In [ ]:
#Only run in colab
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Github/Taskmaster
#!pip install tensorflow --upgrade
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [1]:
import small_rubiks as rubiks
import numpy as np
from student import student
from classroom import classroom
from teacher import teacher
from general_task_network import student_network
from utils import dotdict
from keras import regularizers

task = rubiks.rubiks_task
setup = rubiks.rubiks_setup
rng = np.random.default_rng(seed=0)

core_params = dotdict({
            "residual_weights_reg" : None,
            "residual_bias_reg" : None,
            "relu_leak" : 0.05,
            "residual_units" : 300,
            "learning_rate" : 0.001,
            "residual_layers" : 3
})

value_network_params = dotdict({
            "residual_weights_reg" : None,
            "residual_bias_reg" : None,
            "relu_leak" : 0.05,
            "residual_units" : 300,
            "learning_rate" : 0.001,
            "post_core_residual_layers" : 2,
            "reward_fork_layers": 1,
            "value_fork_layers": 1
})

state_network_params = dotdict({
            "residual_weights_reg" : None,
            "residual_bias_reg" : None,
            "relu_leak" : 0.05,
            "residual_units" : 300,
            "learning_rate" : 0.001,
            "post_core_residual_layers" : 2
})

params = dotdict({
    "core_params":core_params,
    "value_network_params": value_network_params,
    "state_network_params": state_network_params
})

my_student_network = student_network.create(params)
#my_student_network = student_network.load_value_network("models/trained.h5")
student_template = student(task, 30,my_student_network,0.95)
t = teacher(setup, lambda n : 1+rng.poisson(lam=3,size=n))
c = classroom(task, setup, t, student_template, n_students=1, max_steps=10, buffer_size = lambda n : 2048)


In [6]:
c.max_steps=15
t.step_dist = lambda n : 8*np.ones(n,dtype=int)

In [8]:

for i in range(10):
    print(f"Round {i}.")
    c.run_training_batch(n_problems=100,epochs_per_episode=3)
    #if i % 10 == 9:
    #    my_student_network.save("models/trained.h5")

Round 0.
Before step 1, 100 out of 100 remain open.
Before step 2, 100 out of 100 remain open.
Before step 3, 91 out of 100 remain open.
Before step 4, 91 out of 100 remain open.
Before step 5, 68 out of 100 remain open.
Before step 6, 68 out of 100 remain open.
Before step 7, 54 out of 100 remain open.
Before step 8, 54 out of 100 remain open.
Before step 9, 48 out of 100 remain open.
Before step 10, 48 out of 100 remain open.
Before step 11, 46 out of 100 remain open.
Before step 12, 46 out of 100 remain open.
Before step 13, 46 out of 100 remain open.
Before step 14, 46 out of 100 remain open.
Before step 15, 46 out of 100 remain open.
After step 15, 46 out of 100 remain open.
Epoch 1/3
32/32 [==============================] - 1s 23ms/step - loss: 0.0744 - value_output_loss: 0.0591 - reward_output_loss: 0.0075 - reward_confidence_output_loss: 0.0077
Epoch 2/3
32/32 [==============================] - 1s 24ms/step - loss: 0.0559 - value_output_loss: 0.0387 - reward_output_loss: 0.0086

In [9]:
n=4096
rng = np.random.default_rng(seed=0)
states = np.arange(24)[None,:].repeat(n,axis=0)

for k in range(5):
    for i in range(30):
        _, states = rubiks.task_action(states,rng.choice(a=12,size=n))

    actions = rng.choice(12,size=n)
    _, next_states = rubiks.task_action(states,actions)

    actions = (np.arange(12) == actions[...,None]).astype(float)
    colors = rubiks.make_neural_input(states)
    next_colors = rubiks.make_neural_input(next_states)
    my_student_network.fit_state(colors,actions,next_colors,epochs = 10)

Epoch 1/10


ValueError: in user code:

    File "C:\Users\leove\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\leove\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\leove\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\leove\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\leove\AppData\Roaming\Python\Python310\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\leove\AppData\Roaming\Python\Python310\

In [ ]:
states = np.arange(24)[None,:]
actions = np.array([0])
prediction = my_student_network.predict_state(states,actions)
prediction.reshape(1,24,6), prediction.reshape(1,24,6).argmax(axis=2), rubiks.start_coloring[rubiks.task_action(states,actions)[1][0]]